In [6]:
%matplotlib inline
import pandas as pd
import numpy as np
import os
import re
import joblib
import sys

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pylab as plt
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop,Adam
from keras.utils import np_utils
# LSTM for sequence classification in the IMDB dataset
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense,Input
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
from keras.layers import Embedding, LSTM
import json
data_directory = '../data/'


Using TensorFlow backend.


In [7]:
batch_size = 32
nb_classes = 2
nb_epoch = 5

In [8]:
authors = pd.read_csv(os.path.join(data_directory,'accounts.csv'))

In [9]:
sample = pd.read_csv(os.path.join(data_directory,
                                             'sample_data.csv'))



In [10]:
posts_raw_cleaned = pd.read_csv(os.path.join(data_directory,
                                             'posts_raw_cleaned', 
                                             'posts_raw_cleaned.csv'))

# Drop null

posts_raw_cleaned = posts_raw_cleaned[posts_raw_cleaned['body'].notnull()]

# posts_raw_cleaned = posts_raw_cleaned[0:5000]

texts = list(posts_raw_cleaned['body'].str.lower())

labels = posts_raw_cleaned['total_payout_value']

steem_counts = posts_raw_cleaned['body'].str.lower().str.count('steem')
posts_raw_cleaned['number of steem counts'] = steem_counts

In [11]:
percent = np.percentile(authors['reputation'],95)
top_95_idx = (authors['reputation'] > percent)
whale_list = authors.ix[top_95_idx,'name'].values
posts_raw_cleaned['Whale'] = posts_raw_cleaned['author'].isin(whale_list).astype(int)

In [12]:
languages = []
for language in posts_raw_cleaned['body_language']:
    if (language != '[]') & pd.notnull(language):
        languages.append(json.loads(language)[0]['language'])
    else:
        languages.append('unknown')
        
posts_raw_cleaned['language'] = languages    

posts_raw_cleaned['author_reputation_scaled'] = (posts_raw_cleaned['author_reputation'] + 0.0)/(10**14)
labels = (posts_raw_cleaned['total_payout_value'] >  np.median(posts_raw_cleaned['total_payout_value'])).astype(int)

In [14]:
posts_raw_cleaned.ix[posts_raw_cleaned['language'] != 'en','language']

56             nl
517            pt
540            zh
622            ru
663       unknown
694            no
708       unknown
814       unknown
890            zh
1236           de
1313      unknown
1348           zh
1471      unknown
1473           ko
1496           da
1834           zh
1897           de
1967           de
2040           zh
2073           ru
2096           de
2180           zh
2223           zh
2368      unknown
2549           zh
2562           zh
2608           zh
2767           ru
2994           zh
3003           de
           ...   
110714    unknown
110721         no
110729         de
110756         ru
110759         ru
110767         ko
110775         ru
110822         ru
110833         de
110863         ru
110869    unknown
110902         zh
110903         de
110915         ru
110920         de
110929         ru
110975         de
110977         de
110986         es
110989         ru
111042         es
111045         ko
111101         ha
111109         no
111112    

In [15]:
MAX_SEQUENCE_LENGTH = 600
MAX_NB_WORDS = 5000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2

In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 416017 unique tokens.


without Lowercase : Found 416017 unique tokens

with lowercase: 


In [17]:
from keras.utils.np_utils import to_categorical

In [18]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

('Shape of data tensor:', (108818, 600))
('Shape of label tensor:', (108818, 2))


Link to download : http://nlp.stanford.edu/data/glove.6B.zip

In [19]:
GLOVE_DIR = os.path.join('..','word2vec_models','glove.6B')
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.300d.txt'))
count = 0
for line in f:
    count +=1
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))
print count

Found 400000 word vectors.
400000


In [20]:
len(embeddings_index)

400000

'Found 400000 word vectors.
In [15]:
'

In [22]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


In [23]:
rev_lookup = {}
for key in word_index.keys():
    rev_lookup[word_index[key]] = key 

In [28]:
matched = []
for value in posts_raw_cleaned.ix[110975,'body'].split(' '):
    try:
        word_index[value.lower()]
        matched.append(value)
    except:
        pass


In [29]:
len(matched)

20

In [68]:
posts_for_training.head().index

Int64Index([33698, 74046, 68982, 22541, 94380], dtype='int64')

In [99]:
lengths = posts_raw_cleaned['body'].str.split(' ').apply(len)

In [100]:
lengths.describe()

count    108818.000000
mean        521.984322
std        2136.199282
min           1.000000
25%          60.000000
50%         230.000000
75%         620.000000
max      164431.000000
Name: body, dtype: float64

In [32]:
len(posts_for_training.ix[110975,'body'].split(' '))

21

In [27]:
posts_for_training = posts_raw_cleaned.ix[:,:]
print posts_for_training.ix[110975,'body']

Vielen Dank fr dein Feedback freut mich dass dir mein Post gefallen hat und hoffentlich werden dir die Tipps helfen 


In [36]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

labels = ((posts_raw_cleaned['total_payout_value'] >  np.median(posts_raw_cleaned['total_payout_value']))
          .astype(int)).values


VALIDATION_SPLIT = 0.33
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)


features = posts_raw_cleaned.ix[:,['number of body tags',
                                   'number of body urls',
                                   'number of image urls',
                                   'number of body mentions',
                                   'number of image urls',
                                   'number of youtube urls',
                                   'language',
                                   'author_reputation_scaled',
                                   'number of steem counts']]

features = pd.get_dummies(features)
features = ss.fit_transform(features)
number_of_features = features.shape[1]
x_train_features = features[:-nb_validation_samples]
x_values_features = features[-nb_validation_samples:]


In [18]:
from keras import backend as K
# x_train = K.cast_to_floatx(x_train)
# x_val = K.cast_to_floatx(x_val)



x_train_features = K.cast_to_floatx(x_train_features)
x_values_features = K.cast_to_floatx(x_values_features)





In [23]:
y_train = y_train[0:100]
y_val = y_val[0:100]


x_train = x_train[0:100]
x_val = x_val[0:100]
x_values_features = x_values_features[0:100]
x_train_features = x_train_features[0:100]



In [41]:
import json
import io, json
import keras
from keras.layers import Merge
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs)
        with open("LTSM-2.txt", "w") as text_file:
            text = 'acc, loss, val_acc, val_loss' + '\n'
            for dicts in self.losses:
                for key in dicts.keys():
                    text += str(dicts[key]) + ', '
                text += '\n'
            text_file.write(text)

In [42]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
left_branch = Sequential()
left_branch.add(embedding_layer)
left_branch.add(LSTM(100, return_sequences=False,activation = 'softsign'))
left_branch.add(Dense(2, activation='sigmoid'))

right_branch = Sequential()
right_branch.add(Dense(50, input_dim=number_of_features, activation='relu'))
right_branch.add(Dense(20, activation='relu'))

# Mege branchs
merged = Merge([left_branch, right_branch], mode='concat')

final_model = Sequential()
final_model.add(merged)
final_model.add(Dense(y_train.shape[1], activation='softmax'))

final_model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
                    metrics=['accuracy'],)


In [43]:

callback = LossHistory()

history = final_model.fit([x_train, x_train_features],
                          y_train,callbacks=[callback],
                          validation_data=([x_val, x_values_features],y_val),
                          nb_epoch=20)  # we pass one data array per model input

Train on 100 samples, validate on 100 samples
Epoch 1/20
100/100 [==============================] - 4s - loss: 0.7261 - acc: 0.4800 - val_loss: 0.7850 - val_acc: 0.4600
Epoch 2/20
100/100 [==============================] - 3s - loss: 0.6999 - acc: 0.5400 - val_loss: 0.7863 - val_acc: 0.4600
Epoch 3/20
100/100 [==============================] - 3s - loss: 0.6821 - acc: 0.5500 - val_loss: 0.7895 - val_acc: 0.4600
Epoch 4/20
100/100 [==============================] - 3s - loss: 0.6756 - acc: 0.5500 - val_loss: 0.7967 - val_acc: 0.4600
Epoch 5/20
100/100 [==============================] - 3s - loss: 0.6701 - acc: 0.5400 - val_loss: 0.8055 - val_acc: 0.4700
Epoch 6/20
100/100 [==============================] - 3s - loss: 0.6678 - acc: 0.5400 - val_loss: 0.7920 - val_acc: 0.4700
Epoch 7/20
100/100 [==============================] - 3s - loss: 0.6592 - acc: 0.5400 - val_loss: 0.7825 - val_acc: 0.4700
Epoch 8/20
 64/100 [==================>...........] - ETA: 0s - loss: 0.6513 - acc: 0.5938

KeyboardInterrupt: 

In [ ]:
joblib.dump(history.history,os.path.join('../images','first_run' + '_100_history'))


In [ ]:
# embedding_layer = Embedding(len(word_index) + 1,
#                             EMBEDDING_DIM,
#                             weights=[embedding_matrix],
#                             input_length=MAX_SEQUENCE_LENGTH,
#                             trainable=False)
# for W_regularizer in [0, 0.25,0.5,0.75]:
#     for U_regularizer in [0, 0.25,0.5,0.75]:
#         for dropout_W in [0, 0.1,0.25]:
#             for dropout_U in [0,0.1,0.25]:
#                 model2 = Sequential()
#                 model2.add(embedding_layer)
#                 model2.add(LSTM(100, return_sequences=False,
#                                      W_regularizer = W_regularizer,
#                                      U_regularizer = U_regularizer,
#                                      dropout_W = dropout_W,
#                                      dropout_U = dropout_U
#                                ))
#                 model2.add(Dense(2, activation='sigmoid'))
#                 model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#                 history = model2.fit(x_train, y_train, nb_epoch=50, batch_size=32,
#                                      validation_data=(x_val, y_val))
                
#                 params ='_'.join([str(W_regularizer),
#                                   str(U_regularizer),
#                                   str(dropout_W),
#                                   str(dropout_U)])
                
#                 joblib.dump(history.history,os.path.join('../images',params + '_100_history'))

